<a href="https://colab.research.google.com/github/BrunoEiki/Simulacao-Discreta/blob/main/simpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução
Simpy é uma biblioteca que fornece métodos para Escalonar eventos com um mesmo relógio global.

In [1]:
### INSTALAR DEPENDÊNCIAS
!pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Generators
Gera os valores na hora da chamada,  não da  definiçao
Vantagens:
* Menor uso  da memória

Desvantagens:
* maior tempo de processamento
* não dá para  percorrer



In [ ]:
# Notação semelhante ao comp
lista  = (i ** 2  for i in range(0, 10))


## ```yield```
palavra-reservada que funciona como o return
```
x = x +  1
yield x
// na segundaa  chamada, ignora  a parte  do yield 1 e começa daqui
x = 1 + 3
yield x
```
retorna um generator

In [2]:
### IMPORTAÇÃO
import simpy as simp
import statistics as st
import numpy as np

In [ ]:
class Pessoa:
    def __init__(self, env, nome, tempo_estudo, tempo_rede_social):
        self.env = env
        self.nome = nome
        self.tempo_estudo = tempo_estudo
        self.tempo_rede_social = tempo_rede_social
        self.env.process(self.run())
        
    def run(self):
        while True:
            print(f"[{self.nome}] começou a estudar em:", self.env.now)
            yield self.env.timeout(self.tempo_estudo)

            print(f"[{self.nome}] foi para as redes sociais em:", self.env.now)
            yield self.env.timeout(self.tempo_rede_social)


In [ ]:
#@title oi
env = simp.Environment()
p1 = Pessoa(env, "Adonis", 4, 5)
p2 = Pessoa(env, "Bario", 2, 6)
p3 = Pessoa(env, "Cesar", 6, 3)

env.run(until=100)


[17/maio/2023]
# Events

In [ ]:
class Carro:
    def __init__(self, env, modelo, frequencia):
        self.env = env
        self.modelo = modelo
        self.frequencia = frequencia
        self.env.process(self.run())
        
    def run(self):
        while True:
            print(f"[{self.modelo}] passou em:", self.env.now)
            yield self.env.timeout(self.frequencia)


class Aviao:
    def __init__(self, env, modelo, frequencia):
        self.env = env
        self.modelo = modelo
        self.frequencia = frequencia
        self.env.process(self.run())
        
    def run(self):
        while True:
            print(f"[{self.modelo}] passou em:", self.env.now)
            yield self.env.timeout(self.frequencia)


class Barco:
    def __init__(self, env, modelo, frequencia):
        self.env = env
        self.modelo = modelo
        self.frequencia = frequencia
        self.env.process(self.run())
        
    def run(self):
        while True:
            print(f"[{self.modelo}] passou em:", self.env.now)
            yield self.env.timeout(self.frequencia)


In [ ]:
#@title 
env = simp.Environment()
p1 = Carro(env, "Fusca", 4)
p2 = Aviao(env, "Boing 717", 23)
p3 = Barco(env, "Going Merry", 12)

env.run(until=88)

[Fusca] passou em: 0
[Boing 717] passou em: 0
[Going Merry] passou em: 0
[Fusca] passou em: 4
[Fusca] passou em: 8
[Going Merry] passou em: 12
[Fusca] passou em: 12
[Fusca] passou em: 16
[Fusca] passou em: 20
[Boing 717] passou em: 23
[Going Merry] passou em: 24
[Fusca] passou em: 24
[Fusca] passou em: 28
[Fusca] passou em: 32
[Going Merry] passou em: 36
[Fusca] passou em: 36
[Fusca] passou em: 40
[Fusca] passou em: 44
[Boing 717] passou em: 46
[Going Merry] passou em: 48
[Fusca] passou em: 48
[Fusca] passou em: 52
[Fusca] passou em: 56
[Going Merry] passou em: 60
[Fusca] passou em: 60
[Fusca] passou em: 64
[Fusca] passou em: 68
[Boing 717] passou em: 69
[Going Merry] passou em: 72
[Fusca] passou em: 72
[Fusca] passou em: 76
[Fusca] passou em: 80
[Going Merry] passou em: 84
[Fusca] passou em: 84


# 3. RESOURCES
Modelar interação entre processos. 
## 3.1. Resources 
São alocados para as entidades e ficam em estado de "ocupado" enquanto estiver sendo usado. Secretária e médicos são recursos.
## 3.2. Containers
Recursos que, além de serem alocados, também sofrem modificação de quantidade. Por exemplo, combustível é consumido durante abastecimento
## 3.3. Store
Semelhante ao container, porém ao invés de uma quantidade, há alteração de um objeto.

In [ ]:
class Carro:
    def __init__(self, env, nome, tempo_chegada, tempo_abastecimento):
        self.env = env
        self.nome = nome
        self.tempo_chegada = tempo_chegada
        self.tempo_abastecimento = tempo_abastecimento

class Posto:
    def __init__(self, env, bombas: int):
        self.env = env
        self.bombas = bombas
        
    def abastecer(self, carro):
        # espera até que o carro chegue no posto
        yield self.env.timeout(carro.tempo_chegada)
        print(carro.nome, 'chegou em', self.env.now)
        # Solicita pelo uso da bomba. Enquanto está sendo usada
        # não pode ser acessada por outras entidades. 
        with self.bombas.request() as bomba:
            # se a bomba estiver sendo usada por outro
            # carro, esperar até conseguir o acesso
            yield bomba
            print(carro.nome, 'iniciou abastecimento em', self.env.now)
            # tempo de abastecimento
            yield self.env.timeout(carro.tempo_abastecimento)
            print(carro.nome, 'saiu do abastecimento em', self.env.now)

import simpy
env = simpy.Environment()

c1 = Carro(env, 'Lada', 0, 5)        
c2 = Carro(env, 'Fusca', 0, 3)     
c3 = Carro(env, 'Opala', 4, 2)       
c4 = Carro(env, 'Fiat 147', 8, 2)       
c5 = Carro(env, 'Santana', 12, 8)          

# Ao aumentar a capacidade de bombas disponíveis,
# reduz-se o tempo de ociosidade/espera dos carros. O tempo
# final pode ser o mesmo, mas a eficiência melhora.
bombas = simpy.Resource(env, capacity=2)
p = Posto(env, bombas)

env.process(p.abastecer(c1))
env.process(p.abastecer(c2))
env.process(p.abastecer(c3))
env.process(p.abastecer(c4))
env.process(p.abastecer(c5))

env.run()


Lada chegou em 0
Fusca chegou em 0
Lada iniciou abastecimento em 0
Fusca iniciou abastecimento em 0
Fusca saiu do abastecimento em 3
Opala chegou em 4
Opala iniciou abastecimento em 4
Lada saiu do abastecimento em 5
Opala saiu do abastecimento em 6
Fiat 147 chegou em 8
Fiat 147 iniciou abastecimento em 8
Fiat 147 saiu do abastecimento em 10
Santana chegou em 12
Santana iniciou abastecimento em 12
Santana saiu do abastecimento em 20


# 4. Prioridade
* PriorityResource
* PreemptiveResource

In [10]:
def fila(env, recurso, pessoa, chegada, prioridade):
    yield env.timeout(chegada)
    with recurso.request(priority=prioridade) as req:
        yield req
        print(f"{pessoa} atendida em {env.now}")
        yield env.timeout(5)

env = simp.Environment()
# recurso = simp.PriorityResource(env, capacity=1)
recurso = simp.PreemptiveResource(env, capacity=1)

env.process(fila(env, recurso, "Maria", 0, 1000))
env.process(fila(env, recurso, "Paula", 1, 1.2))
env.process(fila(env, recurso, "Alana", 2, 1))
env.process(fila(env, recurso, "Ana", 0, -200000))

env.run()

Maria atendida em 0
Ana atendida em 5
Alana atendida em 10
Paula atendida em 15
